### ライブラリのimport

In [461]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
%matplotlib inline
from preprocessing.initial_preprocessing import initial_preprocessing
from preprocessing.drop_emergency_suspicious_height_weight import drop_emergency_suspicious_height_weight
from preprocessing.calculate_bmi_bmr_category import calculate_bmi_bmr_category

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 解析用の元データを読み込み

In [452]:
df = pd.read_excel('train_data.xlsx', index_col=False)
df.head(2)

,ACCESSIONNO,scan protocol,scan series,target region,scanning length,kV,mean mA,max mA,pitch factor,nomial total collimation width,...,部位名称,プリセット名称,検査方法名称,検査責任者名称,検査/撮影情報01,検査/撮影情報02,撮影機種,検査方法,adult_child,study_date
0,1103165320210101,5.3 QQ Chest - Pelvis Routine,NaN,Chest,862.18,100,594.11,700,1.38,80.0,...,胸部〜骨盤CT,256列 【枠外】胸〜骨盤CT（単純）,単純,加藤大貴,NaN,2.0,Revolution,単純,成人,202101
1,1103165220210101,1.5 QQ Brain-Head Routine TFI-H,NaN,Head,240.75,100,387.65,515,0.52,40.0,...,脳CT,256列 【枠外】脳・頭部CT（単純）,単純,加藤大貴,NaN,2.0,Revolution,単純,成人,202101


## データを前処理
* カラム名の変更
* 複数回スキャンについては,Mean CTDIvolが最大のみ残す
* 救急科などの怪しい身長と体重のデータを削除する

In [453]:
initial_preprocessing(df)
df = remove_duplicate_ctdi(df)
df.reset_index(drop=True, inplace=True)
# df_allで後で、身長体重を代入する分を分けられるようにしておく
df_all = df
df = drop_emergency_suspicious_height_weight(df)

正常に処理が行われました。


### 身長・体重の最終確認: この部分は自分で確認しないと危険

In [454]:
df['height_cm'].sort_values()

1200    101.0
97      101.0
2241    101.0
2362    101.0
1011    101.0
        ...  
8610    190.9
7978    190.9
790     193.0
8       193.0
5567    193.0
Name: height_cm, Length: 8958, dtype: float64

In [455]:
index = df['height_cm'].sort_values(ascending=True)[:30].index.to_list()

In [456]:
df[['scanning length', 'height_cm', 'weight_kg', 'id', 'study_date', 'target region', 'Mean CTDIvol']].loc[index]

,scanning length,height_cm,weight_kg,id,study_date,target region,Mean CTDIvol
1200,796.77,101.0,39.80,75206,2021-06-18,Chest,6.70
97,626.53,101.0,39.80,75206,2021-01-14,Abdomen,12.78
2241,852.06,101.0,39.80,75206,2021-04-21,Chest,6.45
2362,621.51,101.0,39.80,75206,2021-06-02,Abdomen,17.10
1011,796.77,101.0,39.80,75206,2021-05-18,Chest,7.38
265,567.16,101.0,39.80,75206,2021-02-04,Chest,6.03
620,827.20,101.0,39.80,75206,2021-03-29,Chest,6.13
6436,506.63,117.7,76.40,9326611,2023-02-10,Bronchus,10.38
3529,531.81,117.7,76.40,9326611,2022-01-21,Bronchus,9.90
106,456.72,117.7,76.40,9326611,2021-01-15,Chest,9.14


In [457]:
# 最終的な怪しいデータに関しては直接データを確認して削除する
drop_ids = [75206, 9326611, 17049]
for ids in drop_ids:
    df = df[~(df['id'] == ids)]
df.reset_index(drop=True, inplace=True)

### 身長体重予測、線量予測、異常検知でも使用しない項目を削除する

In [458]:
unuse_cols = ['hospital_ward', 'modality', 'adult_child', 'department', 'preset_name']
df.drop(labels=unuse_cols, axis=1, inplace=True)

In [ ]:
df.to_excel('preprocess_all_scan_data.xlsx')

### Pandas_Profilling

In [462]:
profile= ProfileReport(df, title='preprocessing_data')
profile.to_file('preprocessing_data.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:73: UserWarning: Glyph 21336 (\N{CJK UNIFIED IDEOGRAPH-5358}) missing from current font.
  plt.savefig(
/opt/anaconda3/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:73: UserWarning: Glyph 32020 (\N{CJK UNIFIED IDEOGRAPH-7D14}) missing from current font.
  plt.savefig(
/opt/anaconda3/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:73: UserWarning: Glyph 36896 (\N{CJK UNIFIED IDEOGRAPH-9020}) missing from current font.
  plt.savefig(
/opt/anaconda3/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:73: UserWarning: Glyph 24433 (\N{CJK UNIFIED IDEOGRAPH-5F71}) missing from current font.
  plt.savefig(
/opt/anaconda3/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:73: UserWarning: Glyph 12511 (\N{KATAKANA LETTER MI}) missing from current font.
  plt.savefig(
/opt/anaconda3/lib/python3.9/site-packages/pandas_profiling/visualisation

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### 新たな特徴量を作成する

* BMI(Body Mass Index)  
  $BMI = weight(kg) / height(m)^2$  
  
* BMR(Basal Metabolic Rate: 基礎代謝率): 年齢、性別、身長、体重に基づいて計算される。ハリス・ベネディクト方程式と呼ばれる  
  男性$BMR=66.47 + (13.75*weight[kg]) + (5.003*height[cm]) - (6.755*age[歳])$  
  女性$BMR=655.1 + (9.563*weight[kg]) + (1.85*height[cm]) - (4.676*age[歳])$  
  
* 体格指数(Rohrer's index) ：学童期に適するとあるので、今回は使わない  
  $Rohrer's index = weight[kg] / height[m]^3$

体格指数（BMI）: 体重（kg）を身長（m）の二乗で割ったもので、以下のように分類されます（成人向けのWHOのガイドライン）。

18.5未満: 低体重（痩せ型）: underweight    
18.5～24.9: 標準体重: normal weight   
25.0～29.9: 肥満（1度）:obesity class1  
30.0～34.9: 肥満（2度）:obesity class2  
35.0～39.9: 肥満（3度）:obesity class3    
40以上: 肥満（4度、重症）:obesity class4

In [450]:
from preprocessing.calculate_bmi_bmr_category import calculate_bmi_bmr_category

In [459]:
calculate_bmi_bmr_category(df)

### 前処理が終了したデータを保存する。

In [493]:
df.to_excel('preprocessed_train_data.xlsx', index=False)